99% Accuracy in 30 Minutes
==============

Using the following code, I was able to achieve a score 99.071% on the Kaggle digit recognizer challenge, with a total training time of roughly 30 minutes on my Core i5 MacBook Pro. As of writing this, I'm sitting in 47th place overall.

In [ ]:
import pyneural
import numpy as np
import pandas as pd
import scipy as sp
import scipy.ndimage as nd

In [ ]:
# load the training data and transform it into usable numpy arrays
# Note: you may need to change the path to the data
training_set = pd.read_csv('~/kaggle/digits/train.csv')
labels = np.array(training_set)[:, 0]
features = np.array(training_set)[:, 1:].astype(float) / 255.0

n_rows = features.shape[0]
n_features = features.shape[1]
n_labels = 10
n_steps = 5

labels_expanded = np.zeros((n_rows, n_labels))
for i in xrange(n_rows):
    labels_expanded[i][labels[i]] = 1

Expanding the training set is the key to success
---------------------------------------------

I had managed to get over 98% accuracy on the test set with PyNeural in the past, but my ability to crack 99% was thanks to insight gained from [this excellent blog post](http://nicklocascio.com/neural-net-mnist-kaggle/).

The MNIST training set supplied in the Kaggle competition is, by computer vision standards, relatively small, so overfitting is a big issue. Nick Locascio points out in his blog post above that you can greatly, and almost trivially, expand the training set using simple transformations of the given training examples.

Nick uses vertical and horizontal shifts, or "nudges", and rotations to multiply the number of training examples. I used the `scipy.ndimage` to perform these transformations, and further expanded on these by adding horizontal and vertical scalings as well. 

You could increase the training set further by repeating these transformations with different degrees of shifting, rotation, and scaling; I would imagine there is some point of marginal returns, but I don't actually know where that is.

In [ ]:
# transform the training set into 42000 28x28 pixel "images"
ims = features.reshape((42000 , 28, 28))

# shift each image down, up, right, and left, respectively, by 2 pixels
dshift = nd.shift(ims, (0, 2, 0), order=0).reshape((42000, 784))
ushift = nd.shift(ims, (0, -2, 0), order=0).reshape((42000, 784))
rshift = nd.shift(ims, (0, 0, 2), order=0).reshape((42000, 784))
lshift = nd.shift(ims, (0, 0, -2), order=0).reshape((42000, 784))

In [ ]:
# rotate each image by 15 degrees both counter-clockwise and clockwise
lrotate = nd.rotate(ims, 15, axes=(1,2), reshape=False, prefilter=False).reshape((42000, 784))
rrotate = nd.rotate(ims, -15, axes=(1,2), reshape=False, prefilter=False).reshape((42000, 784))

In [ ]:
# scale each image by 1.5 in both the vertical and horizontal directions
vscale = nd.zoom(ims, (1, 1.5, 1), order=0, prefilter=False)[:, 7:-7, :].reshape((42000, 784))
hscale = nd.zoom(ims, (1, 1, 1.5), order=0, prefilter=False)[:, :, 7:-7].reshape((42000, 784))

In [ ]:
# combine each of the transformations along with the original training set into a super set
new_features = np.vstack((features, dshift, ushift, rshift, lshift, lrotate, rrotate, vscale, hscale))
new_labels = np.vstack(9 * [labels_expanded])

Training the Neural Net
----------------------

To achieve my results, I used a neural network with two hidden layers of 400 nodes each and trained it over 65 epochs using mini-batches of size 100, a learning rate of 0.01, no L2 regularization, and no decay of the learning rate.

In [ ]:
nn = pyneural.NeuralNet([784, 400, 400, 10])
nn.train(new_features, new_labels, 65, 100, 0.01, 0.0, 1.0)

In [ ]:
# check the accuracy on the training set
preds = nn.predict_label(new_features)
correct = np.sum(preds == np.hstack(9 * [labels]))        
print "%f%% percent correct " % (100.0 * correct / new_features.shape[0])

In [ ]:
# load the test set and make our predictions
test_set = pd.read_csv('~/kaggle/digits/test.csv')
test_features = np.array(test_set).astype(float) / 255.0
test_preds = nn.predict_label(test_features)

In [ ]:
# save our predictions to a csv file
df = pd.DataFrame({'ImageId': np.arange(1, len(test_preds) + 1), 'Label': test_preds})
df.to_csv('/Users/taylor/kaggle/digits/exp_set4.csv', index=False)